# XRT Blop Demo

In [ ]:
from blop.utils import prepare_re_env  # noqa
%run -i $prepare_re_env.__file__ --db-type=temp
bec.disable_plots()

Exception: File `'prepare_re_env.__file__'` not found.

In [ ]:
import sys, os
os.environ["EPICS_CA_ADDR_LIST"] = "127.0.0.1"
os.environ["EPICS_CA_AUTO_ADDR_LIST"] = "NO"
from matplotlib import pyplot as plt
from xrt_beamline import Beamline

from blop import DOF, Objective, Agent
from blop.digestion import beam_stats_digestion
import time
sys.path.append(os.getcwd())

In [ ]:
plt.ion()

h_opt = 0
dh = 5

R1, dR1 = 40000, 10000
R2, dR2 = 20000, 10000

bec.disable_plots()

In [ ]:
beamline = Beamline(name="bl")
time.sleep(1)
dofs = [
    DOF(description="KBV R",
        device=beamline.kbv_dsv,
        search_domain=(R1-dR1, R1+dR1)),
    DOF(description="KBH R",
        device=beamline.kbh_dsh,
        search_domain=(R2-dR2, R2+dR2)),

]

In [ ]:
objectives = [
    Objective(name="bl_det_sum", 
              target="max",
              transform="log",
              trust_domain=(20, 1e12)),

    Objective(name="bl_det_wid_x",
              target="min",
              transform="log",
              # trust_domain=(0, 1e12),
              latent_groups=[("bl_kbh_dsh", "bl_kbv_dsv")]),
    Objective(name="bl_det_wid_y",
              target="min",
              transform="log",
              # trust_domain=(0, 1e12),
              latent_groups=[("bl_kbh_dsh", "bl_kbv_dsv")]),
]

In [ ]:
agent = Agent(
    dofs=dofs,
    objectives=objectives,
    detectors=[beamline.det],
    digestion=beam_stats_digestion,
    digestion_kwargs={"image_key": "bl_det_image"},
    verbose=True,
    db=db,
    tolerate_acquisition_errors=False,
    enforce_all_objectives_valid=True,
    train_every=3,
)

In [ ]:
RE(agent.learn("qr", n=16))
RE(agent.learn("qei", n=16, iterations=4))


In [ ]:
agent.plot_objectives(axes=(0, 1))